In [1]:
import tensorflow as tf
import numpy as np

Solving the problem
\begin{eqnarray}
\left[\begin{matrix}
3 & 0\\
1 & 2\\
0 & 0
\end{matrix}\right]
\left[\begin{matrix}
w_1\\
w_2
\end{matrix}\right]=
\left[\begin{matrix}
0\\
0\\
1
\end{matrix}\right],
\end{eqnarray}
Obviously, this equation has no accurate solution. So try to solve it using lasso, using some descent method in tensorflow:
$$
\mathop{\text{min}}\limits_{w}~\|Aw-b\|_2^2+\lambda \|w\|_1^2
$$
with
$$
A=
\left[\begin{matrix}
3 & 0\\
1 & 2\\
0 & 0
\end{matrix}\right],
b=
\left[\begin{matrix}
0\\
0\\
1
\end{matrix}\right],
$$
and with $\lambda$ given (for example, $\lambda=.001$).

In [2]:
learning_rate=0.01
lam=0.001
training_epoch=100
display_step=20

In [3]:
train_features=np.asarray([[3.0, 0.0], [1.0, 2.0], [0.0, 0.0]])
train_labels=np.asarray([[0], [0], [1]])
n_samples=train_features.shape[0]
print n_samples

3


In [4]:
# deletable
print zip(train_features, train_labels)[0]
print zip(train_features, train_labels)[1]
print zip(train_features, train_labels)[2]

(array([3., 0.]), array([0]))
(array([1., 2.]), array([0]))
(array([0., 0.]), array([1]))


In [5]:
# deletable
print zip(train_features, train_labels)[0][0]
print zip(train_features, train_labels)[0][1]
print train_features

[3. 0.]
[0]
[[3. 0.]
 [1. 2.]
 [0. 0.]]


In [6]:
# A=tf.placeholder("float32",[None, 2])
# b=tf.placeholder("float32",[None, 1])
A=tf.placeholder("float32")
b=tf.placeholder("float32")

# least square solution

In [15]:
print 'W_true=',np.dot(np.linalg.pinv(train_features), train_labels)
print np.dot(np.linalg.pinv(train_features), train_features)

W_true= [[0.]
 [0.]]
[[1.00000000e+00 1.11022302e-16]
 [1.11022302e-16 1.00000000e+00]]


# A big error caused by the following difference

## right answer!

In [7]:
W=tf.Variable(np.random.randn(2), name="weight", dtype="float32")
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print sess.run(W)

[-1.3357573 -0.2138889]


In [8]:
predict=tf.multiply(A, W)
cost=tf.reduce_sum(tf.pow(predict-b, 2))+lam*tf.pow(tf.norm(W,1),2)
optimizer=tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

#### Method-1: Each training use a single example

In [9]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(training_epoch):
        for (x, y) in zip(train_features, train_labels):
            sess.run(optimizer, feed_dict = {A: x, b: y})
            if (epoch+1) % display_step == 0:
                print 'epoch=',epoch+1, 'cost=', sess.run(cost, feed_dict={A: x, b: y}), 'weight=',sess.run(W)
                print 'training examples is:', (x, y)

epoch= 20 cost= 0.002651428 weight= [-0.01715202 -0.0437058 ]
training examples is: (array([3., 0.]), array([0]))
epoch= 20 cost= 0.0067525227 weight= [-0.01680776 -0.04020812]
training examples is: (array([1., 2.]), array([0]))
epoch= 20 cost= 2.0000033 weight= [-0.01680662 -0.04020698]
training examples is: (array([0., 0.]), array([1]))
epoch= 40 cost= 4.787501e-07 weight= [-0.00021275 -0.00823528]
training examples is: (array([3., 0.]), array([0]))
epoch= 40 cost= 0.00022970482 weight= [-0.00020833 -0.00757629]
training examples is: (array([1., 2.]), array([0]))
epoch= 40 cost= 2.0 weight= [-0.00020817 -0.00757614]
training examples is: (array([0., 0.]), array([1]))
epoch= 60 cost= 2.4532831e-09 weight= [-2.059159e-06 -1.552007e-03]
training examples is: (array([3., 0.]), array([0]))
epoch= 60 cost= 8.156675e-06 weight= [-1.9868944e-06 -1.4278154e-03]
training examples is: (array([1., 2.]), array([0]))
epoch= 60 cost= 2.0 weight= [-1.9582983e-06 -1.4277868e-03]
training examples is:

#### Method-2: Each training use `batch_size = 2` examples

See `https://github.com/suzyi/tensorflow/blob/master/tf/lasso-datasetAPI.ipynb` for Method-2.

#### Method-3: Each training use the full dataset

In [10]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(training_epoch):
        sess.run(optimizer, feed_dict = {A: train_features, b: train_labels})
        if (epoch+1) % display_step == 0:
            print 'epoch=',epoch+1, 'cost=', sess.run(cost, feed_dict={A: train_features, b: train_labels}), 'weight=',sess.run(W)
            print 'training examples is:', (x, y)

epoch= 20 cost= 2.0088675 weight= [-0.01538623 -0.04030244]
training examples is: (array([0., 0.]), array([1]))
epoch= 40 cost= 2.0002315 weight= [-0.00017611 -0.00760104]
training examples is: (array([0., 0.]), array([1]))
epoch= 60 cost= 2.000008 weight= [-1.8050894e-06 -1.4336377e-03]
training examples is: (array([0., 0.]), array([1]))
epoch= 80 cost= 2.0000002 weight= [ 3.5589729e-09 -2.7040028e-04]
training examples is: (array([0., 0.]), array([1]))
epoch= 100 cost= 2.0 weight= [ 5.9782235e-10 -5.1000570e-05]
training examples is: (array([0., 0.]), array([1]))


## wrong answer!
- caused by the dimension `(2, 1)` in `W=tf.Variable(np.random.randn(2, 1), name="weight", dtype="float32")`

In [11]:
W=tf.Variable(np.random.randn(2, 1), name="weight", dtype="float32")
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print sess.run(W)

[[-2.0533164]
 [ 0.1464007]]


In [12]:
predict=tf.multiply(A, W)
cost=tf.reduce_sum(tf.pow(predict-b, 2))+lam*tf.pow(tf.norm(W,1),2)
optimizer=tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [13]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(training_epoch):
        for (x, y) in zip(train_features, train_labels):
            sess.run(optimizer, feed_dict = {A: x, b: y})
            if (epoch+1) % display_step == 0:
                print 'epoch=',epoch+1, 'cost=', sess.run(cost, feed_dict={A: x, b: y}), 'weight=',sess.run(W)
                print 'training examples is:', (x, y)

epoch= 20 cost= 0.00024768684 weight= [[-0.00523288]
 [ 0.00036639]]
training examples is: (array([3., 0.]), array([0]))
epoch= 20 cost= 0.00011146473 weight= [[-0.00470948]
 [ 0.00032964]]
training examples is: (array([1., 2.]), array([0]))
epoch= 20 cost= 4.0 weight= [[-0.00470938]
 [ 0.00032953]]
training examples is: (array([0., 0.]), array([1]))
epoch= 40 cost= 1.3026883e-09 weight= [[-1.2001884e-05]
 [ 8.2441130e-07]]
training examples is: (array([3., 0.]), array([0]))
epoch= 40 cost= 5.862394e-10 weight= [[-1.0801439e-05]
 [ 7.4171368e-07]]
training examples is: (array([1., 2.]), array([0]))
epoch= 40 cost= 4.0 weight= [[-1.0801208e-05]
 [ 7.4148284e-07]]
training examples is: (array([0., 0.]), array([1]))
epoch= 60 cost= 6.8514344e-15 weight= [[-2.7527006e-08]
 [ 1.8543296e-09]]
training examples is: (array([3., 0.]), array([0]))
epoch= 60 cost= 3.0833014e-15 weight= [[-2.4773719e-08]
 [ 1.6683090e-09]]
training examples is: (array([1., 2.]), array([0]))
epoch= 60 cost= 4.0 wei